<a href="https://colab.research.google.com/github/Amal-David/AD/blob/master/T1_T2_Orchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# initialization and Connection to DB
import pymongo
from pymongo import MongoClient
import requests
import json
import time
import datetime
from pytz import timezone
import pytz
import decimal

Indian = timezone('Asia/Kolkata')
client = MongoClient('mongodb://13.232.240.53:15593/')
db = client.oman_pilot
telemetry = db.telemetry_data

# Pulling data from UI
values = db.configuration
# levels = db.water_level

logs = db.logger


def log_func(tank, trigger, action):
    timestamp = datetime.datetime.now()
    timestamp = timestamp.astimezone(Indian)
    initial_config = list(values.find({}))
    final_t1_water_level = initial_config[-1]['leg1_t1_current_water_level']
    final_t2_water_level = initial_config[-1]['leg1_t2_current_water_level']
    # final_t3_water_level = initial_config[-1]['t3_current_water_level']
    t1_pH = telemetry.find_one({"dev_id": "ffffffff25274919"}, {"payload_fields.pH": 1},
                               sort=[('_id', pymongo.DESCENDING)])
    t1_pH = float(t1_pH['payload_fields']['pH'])

    t1_TDS_ppm = telemetry.find_one({"dev_id": "ffffffff25274478"}, {"payload_fields.TDS_ppm": 1},
                                    sort=[('_id', pymongo.DESCENDING)])
    t1_TDS_ppm = float(t1_TDS_ppm['payload_fields']['TDS_ppm'])

    t1_EC1_uS_per_cm = telemetry.find_one({"dev_id": "ffffffff25274478"}, {"payload_fields.EC_mS_per_cm": 1},
                                          sort=[('_id', pymongo.DESCENDING)])
    t1_EC1_uS_per_cm = float(t1_EC1_uS_per_cm['payload_fields']['EC_mS_per_cm'])

    t2_Humidity = list(telemetry.find({"dev_id": "lst25275916"}, {"payload_fields.Humidity": 1}))
    t2_Humidity = float(t2_Humidity[-1]['payload_fields']['Humidity'])

    t2_Temperature = list(telemetry.find({"dev_id": "lst25275916"}, {"payload_fields.Temperature": 1}))
    t2_Temperature = float(t2_Temperature[-1]['payload_fields']['Temperature'])

    t2_pH = list(telemetry.find({"dev_id": "ffffffff25274475"}, {"payload_fields.pH": 1}))
    t2_pH = float(t2_pH[-1]['payload_fields']['pH'])

    t2_TDS_ppm = list(telemetry.find({"dev_id": "ffffff0025274924"}, {"payload_fields.TDS_ppm": 1}))
    t2_TDS_ppm = float(t2_TDS_ppm[-1]['payload_fields']['TDS_ppm'])

    t2_EC1_uS_per_cm = list(telemetry.find({"dev_id": "ffffff0025274924"}, {"payload_fields.EC_mS_per_cm": 1}))
    t2_EC1_uS_per_cm = float(t2_EC1_uS_per_cm[-1]['payload_fields']['EC_mS_per_cm'])

    water_level = "T1 - " + str(final_t1_water_level) + " T2 - " + str(final_t2_water_level)
    telemetry_values = "T1 - pH " + str(t1_pH) + " T1 - EC " + str(t1_EC1_uS_per_cm) + " T1 - TDS " + str(
        t1_TDS_ppm) + " T2 - pH " + str(t2_pH) + " T2 - EC " + str(t2_EC1_uS_per_cm) + " T2 - TDS " + str(
        t2_TDS_ppm) + " T2 - Temperature " + str(t2_Temperature) + " T2 - Humidity " + str(t2_Humidity)
    data_id1 = logs.insert_one({"Tank": tank, "Trigger": trigger, "Action": action, "Result": water_level,
                                "Telemetry_Values": telemetry_values, "Timestamp": timestamp})
    return


# t1_initial_water_level = 70
# t2_initial_water_level = 160
# t3_initial_water_level = 75

initial_config = list(values.find({}))
leg1_t1_refill = initial_config[-1]['leg1_t1_refill']
leg1_water_flow_to_t2 = initial_config[-1]['leg1_water_flow_to_t2']
leg1_pump_1_delay = initial_config[-1]['leg1_pump_1_delay']
leg1_t2_pH_max = initial_config[-1]['leg1_t2_pH_max']
leg1_t2_pH_min = initial_config[-1]['leg1_t2_pH_min']
leg1_t2_Humidity = initial_config[-1]['leg1_t2_Humidity']
leg1_t2_Temperature = initial_config[-1]['leg1_t2_Temperature']
leg1_t2_Humidity_max = initial_config[-1]['leg1_t2_Humidity_max']
leg1_t2_Humidity_min = initial_config[-1]['leg1_t2_Humidity_min']
leg1_t2_Temperature_max = initial_config[-1]['leg1_t2_Temperature_max']
leg1_t2_Temperature_min = initial_config[-1]['leg1_t2_Temperature_min']
leg1_t1_current_water_level = initial_config[-1]['leg1_t1_current_water_level']
t1_initial_water_level = leg1_t1_current_water_level  # taking water level from db
leg1_t1_initial_water_level = initial_config[-1]['leg1_t1_initial_water_level']
leg1_t1_t2_pump_delay = initial_config[-1]['leg1_t1_t2_pump_delay']
leg1_t1_t2_pump_status = initial_config[-1]['leg1_t1_t2_pump_status']
leg1_t2_current_water_level = initial_config[-1]['leg1_t2_current_water_level']
t2_initial_water_level = leg1_t2_current_water_level  # taking water level from db
leg1_t2_t1_pump_delay = initial_config[-1]['leg1_t2_t1_pump_delay']
leg1_t2_t3_pump_delay = initial_config[-1]['leg1_t2_t3_pump_delay']
leg1_water_flow_t1_t2 = initial_config[-1]['leg1_water_flow_t1_t2']
leg1_water_flow_t2_t1 = initial_config[-1]['leg1_water_flow_t2_t1']
leg1_water_flow_t2_t3 = initial_config[-1]['leg1_water_flow_t2_t3']
# t3_current_water_level = initial_config[-1]['t3_current_water_level']

# solenoid t1
solenoid_valve_t1_downlink = 'https://integrations.cultyvate.thethings.industries/ttn/api/v2/down/003602002f1/aws_lambda?key=cultyvate-account.gfLj4ayXM8SPoTMo2sFVIbNqx47LaAnHyCHvMJNBW4A'

# pump downlinks
t1_t2_pump_downlink = 'https://integrations.cultyvate.thethings.industries/ttn/api/v2/down/ph_single_phase_pump/aws_lambda?key=cultyvate-account.M3tbZRg_L89OURpu6MtthueRr1l0FdQKUQkhetwHdUk'
t2_t1_pump_downlink = 'https://integrations.cultyvate.thethings.industries/ttn/api/v2/down/ph_single_phase_pump/aws_lambda?key=cultyvate-account.M3tbZRg_L89OURpu6MtthueRr1l0FdQKUQkhetwHdUk'
t2_t3_pump_downlink = 'https://integrations.cultyvate.thethings.industries/ttn/api/v2/down/ec_tds_single_phase_pump/aws_lamda?key=cultyvate-account.gOflW1gFoWiQ9bOtqZihsfho7XXPlpHHBRfRadAtbJY'

# Initializing temp flags and a bunch of variablesf
temp_t1_solenoid_status = 0
temp_t1_t2_pump_status = 0
temp_t2_t3_pump_status = 0
temp_t3_pump_status = 0
temp_t3_farm_solenoid_status = 0
temp_t3_drain_solenoid_status = 0
temp_t1_t2_status = 0
temp_t2_t1_status = 0
temp_t2_t3_status = 0
temp_t3_drain_solenoid_status = 0
initial_run = 1
water_flown_to_t2 = 0
water_flown_to_t1 = 0
water_flown_to_t3 = 0
t1_valve_command_sent = 0
temp_t1_pump_flag = 0
t2_t3_counter = 0
first_run = 1
recirculation_flag = 0

t1_solenoid_on_query = {
    "dev_id": "a840410001818da9",
    "port": 2,
    "confirmed": True,
    "payload_fields": {"Sampling_Freq": 0, "Valve_Control": 2}
}

t1_solenoid_off_query = {
    "dev_id": "a840410001818da9",
    "port": 2,
    "confirmed": True,
    "payload_fields": {"Sampling_Freq": 0, "Valve_Control": 1}
}


def t1_to_t2_pump_on():
    config_data_for_downlink = list(values.find({}))
    leg1_water_flow_to_t2 = config_data_for_downlink[-1]['leg1_water_flow_to_t2']
    leg1_pump_1_delay = config_data_for_downlink[-1]['leg1_pump_1_delay']
    t1_to_t2_pump_run_time = (leg1_water_flow_to_t2 * 60) / 2.4  # formula for calculating pump run time in seconds for given water
    t1_to_t2_pump_on_query = {
        "dev_id": "ffffffff25274919",
        "port": 2,
        "confirmed": True,
        "payload_fields": {"sampling_freq": 0, "delay_time": leg1_pump_1_delay, "pump_run_time": t1_to_t2_pump_run_time,
                           "DN_restrict_time": 0, "on_off": 1}}

    requests.post(t1_t2_pump_downlink, data=json.dumps(t1_to_t2_pump_on_query))



def t1_t2_pump_on():
    config_data_for_downlink = list(values.find({}))
    leg1_t1_t2_pump_delay = config_data_for_downlink[-1]['leg1_t2_t1_pump_delay']
    leg1_water_flow_t1_t2 = config_data_for_downlink[-1]['leg1_water_flow_t2_t1']
    t1_t2_pump_run_time = (leg1_water_flow_t1_t2 * 60) / 2.4  # formula for calculating pump run time in seconds for given water
    t1_t2_pump_on_query = {
        "dev_id": "ffffffff25274919",
        "port": 2,
        "confirmed": True,
        "payload_fields": {"sampling_freq": 0, "delay_time": leg1_t1_t2_pump_delay,
                           "pump_run_time": t1_t2_pump_run_time, "DN_restrict_time": 0, "on_off": 1}

    }

    requests.post(t1_t2_pump_downlink, data=json.dumps(t1_t2_pump_on_query))


def t2_t1_pump_on():

    config_data_for_downlink = list(values.find({}))
    leg1_t2_t1_pump_delay = config_data_for_downlink[-1]['leg1_t2_t1_pump_delay']
    leg1_water_flow_t2_t1 = config_data_for_downlink[-1]['leg1_water_flow_t2_t1']
    t2_t1_pump_run_time = (leg1_water_flow_t2_t1 * 60) / 2.4  # formula for calculating pump run time in seconds for given water
    t2_t1_pump_on_query = {
        "dev_id": "ffffffff25274475",
        "port": 2,
        "confirmed": True,
        "payload_fields": {"sampling_freq": 0, "delay_time": leg1_t2_t1_pump_delay, "pump_run_time": t2_t1_pump_run_time,
                           "DN_restrict_time": 0, "on_off": 1}}
    requests.post(t2_t1_pump_downlink, data=json.dumps(t2_t1_pump_on_query))


def t2_t3_pump_on():

    config_data_for_downlink = list(values.find({}))
    leg1_water_flow_t2_t3 = config_data_for_downlink[-1]['leg1_water_flow_t2_t3']
    leg1_t2_t3_pump_delay = config_data_for_downlink[-1]['leg1_t2_t3_pump_delay']
    t2_t3_pump_run_time = (leg1_water_flow_t2_t3 * 60) / 2.4  # formula for calculating pump run time in seconds for given water
    t2_t3_pump_on_query = {
        "dev_id": "ffffff0025274924",
        "port": 2,
        "confirmed": True,
        "payload_fields": {"sampling_freq": 0, "delay_time": leg1_t2_t3_pump_delay, "pump_run_time": t2_t3_pump_run_time,
                           "DN_restrict_time": 0, "on_off": 1}
    }
    requests.post(t2_t3_pump_downlink, data=json.dumps(t2_t3_pump_on_query))


def t1_solenoid_valve_on():
    requests.post(solenoid_valve_t1_downlink, data=json.dumps(t1_solenoid_on_query))


def t1_solenoid_valve_off():
    requests.post(solenoid_valve_t1_downlink, data=json.dumps(t1_solenoid_off_query))


# Orchestration starts for tank 1
flow_t1 = list(telemetry.find({"dev_id": "adwl_30000101"}, {"payload_fields.Water_Flow_in_Cubic_Meter": 1}))
t1_start_water_level = (flow_t1[-1]['payload_fields'][
                            'Water_Flow_in_Cubic_Meter'] * 1000) - t1_initial_water_level  # calculating start water level
# print("T1 Start water level ", t1_start_water_level)
flow_t2 = list(telemetry.find({"dev_id": "adwl_30000102"}, {"payload_fields.Water_Flow_in_Cubic_Meter": 1}))
t2_start_water_level = (flow_t2[-1]['payload_fields'][
                            'Water_Flow_in_Cubic_Meter'] * 1000) - t2_initial_water_level  # calculating start water level

T2_orchestrator = 0
move_to_t2 = 0
move_to_t3 = 0
move_to_pH = 0
move_to_farm_drain = 0
temp_static_t1 = 0
temp_t1_solenoid_valve = 0

while True:

    if first_run == 1:
        log_action = "T1 Orchestrator Started"
        log_func("Tank 1", "NA", log_action)
    else:
        pass
    config_data = list(values.find({}))
    leg1_t1_refill = config_data[-1]['leg1_t1_refill']
    leg1_water_flow_to_t2 = config_data[-1]['leg1_water_flow_to_t2']
    leg1_pump_1_delay = config_data[-1]['leg1_pump_1_delay']
    leg1_t2_pH_max = config_data[-1]['leg1_t2_pH_max']
    leg1_t2_pH_min = config_data[-1]['leg1_t2_pH_min']
    leg1_t2_Humidity = config_data[-1]['leg1_t2_Humidity']
    leg1_t2_Temperature = config_data[-1]['leg1_t2_Temperature']
    leg1_t2_Humidity_max = float(config_data[-1]['leg1_t2_Humidity_max'])
    leg1_t2_Humidity_min = float(config_data[-1]['leg1_t2_Humidity_min'])
    leg1_t2_Temperature_max = float(config_data[-1]['leg1_t2_Temperature_max'])
    leg1_t2_Temperature_min = float(config_data[-1]['leg1_t2_Temperature_min'])
    leg1_water_flow_t1_t2 = config_data[-1]['leg1_water_flow_t1_t2']
    leg1_t2_t1_pump_delay = config_data[-1]['leg1_t2_t1_pump_delay']
    leg1_water_flow_t2_t1 = config_data[-1]['leg1_water_flow_t2_t1']
    leg1_water_flow_t2_t3 = config_data[-1]['leg1_water_flow_t2_t3']
    leg1_t2_t3_pump_delay = config_data[-1]['leg1_t2_t3_pump_delay']
    water_flow_t3_farm = config_data[-1]['water_flow_t3_farm']

    t1_pH = telemetry.find_one({"dev_id": "ffffffff25274919"}, {"payload_fields.pH": 1},
                               sort=[('_id', pymongo.DESCENDING)])
    t1_pH = float(t1_pH['payload_fields']['pH'])

    t1_TDS_ppm = telemetry.find_one({"dev_id": "ffffffff25274478"}, {"payload_fields.TDS_ppm": 1},
                                    sort=[('_id', pymongo.DESCENDING)])
    t1_TDS_ppm = float(t1_TDS_ppm['payload_fields']['TDS_ppm'])

    t1_EC1_uS_per_cm = telemetry.find_one({"dev_id": "ffffffff25274478"}, {"payload_fields.EC_mS_per_cm": 1},
                                          sort=[('_id', pymongo.DESCENDING)])
    t1_EC1_uS_per_cm = float(t1_EC1_uS_per_cm['payload_fields']['EC_mS_per_cm'])

    t2_Humidity = list(telemetry.find({"dev_id": "lst25275916"}, {"payload_fields.Humidity": 1}))
    t2_Humidity = float(t2_Humidity[-1]['payload_fields']['Humidity'])

    t2_Temperature = list(telemetry.find({"dev_id": "lst25275916"}, {"payload_fields.Temperature": 1}))
    t2_Temperature = float(t2_Temperature[-1]['payload_fields']['Temperature'])

    t2_pH = list(telemetry.find({"dev_id": "ffffffff25274475"}, {"payload_fields.pH": 1}))
    t2_pH = float(t2_pH[-1]['payload_fields']['pH'])

    t2_TDS_ppm = list(telemetry.find({"dev_id": "ffffff0025274924"}, {"payload_fields.TDS_ppm": 1}))
    t2_TDS_ppm = float(t2_TDS_ppm[-1]['payload_fields']['TDS_ppm'])

    t2_EC1_uS_per_cm = list(telemetry.find({"dev_id": "ffffff0025274924"}, {"payload_fields.EC_mS_per_cm": 1}))
    t2_EC1_uS_per_cm = float(t2_EC1_uS_per_cm[-1]['payload_fields']['EC_mS_per_cm'])

    print('')  # New counter
    initial_run += 1
    print('Loop time', datetime.datetime.now(), " run count ", initial_run)

    # log_func("T1-T2 Orchestrator Info", "NA", "Loop Count " + str(initial_run))

    #set_pump_query_parameters()  # Setting pump query parameters

    # Initializing the current water flow
    flow_t1 = list(telemetry.find({"dev_id": "adwl_30000101"}, {"payload_fields.Water_Flow_in_Cubic_Meter": 1}))
    current_t1_water_level = flow_t1[-1]['payload_fields']['Water_Flow_in_Cubic_Meter'] * 1000
    previous_3_t1_water_level = flow_t1[-3]['payload_fields']['Water_Flow_in_Cubic_Meter'] * 1000
    flow_t2 = list(telemetry.find({"dev_id": "adwl_30000102"}, {"payload_fields.Water_Flow_in_Cubic_Meter": 1}))
    present_t2_water_level = flow_t2[-1]['payload_fields'][
                                 'Water_Flow_in_Cubic_Meter'] * 1000  # calculating start water level
    flow_t3 = list(telemetry.find({"dev_id": "adwl_30000103"}, {"payload_fields.Water_Flow_in_Cubic_Meter": 1}))
    # t3_current_water_level = flow_t3[-1]['payload_fields']['Water_Flow_in_Cubic_Meter'] * 1000  # calculating start water level
    solenoid_valve_t1_status_raw = list(
        telemetry.find({"dev_id": "a840410001818da9"}, {"payload_fields.Operating_Mode": 1}))
    t1_solenoid_status = solenoid_valve_t1_status_raw[-1]['payload_fields']['Operating_Mode']

    print("T1 Solenoid Status = ", t1_solenoid_status)

    present_t1_water_level = current_t1_water_level - t1_start_water_level
    final_t1_water_level = present_t1_water_level - water_flown_to_t2 + water_flown_to_t1  # display this when final_t1_water level is greater than present t1 water level

    final_t2_water_level = config_data[-1]['leg1_t2_current_water_level']
    # present_t2_water_level = t2_current_water_level - t2_start_water_level
    # final_t2_water_level = present_t2_water_level + water_flown_to_t2 - water_flown_to_t1

    # present_t3_water_level = t3_current_water_level - t3_start_water_level
    # final_t3_water_level = water_flown_to_t3 + present_t3_water_level

    log_action = "T1 - ", final_t1_water_level, " T2 - ", final_t2_water_level,
    print(log_action)
    # log_func("Info", "NA", "NA")

    #  log_func("Info", "Current values saved to DB", "") #mention data sent to DB

    # log_func("Info", "NA", "Current values saved to DB")  # mention data sent to DB

    data_id = values.update_one(
        {"id": "ui_values"},
        {
            "$set": {
                "leg1_t2_Temperature_current": t2_Temperature,
                "leg1_t2_pH_current": t2_pH,
                "leg1_t2_EC_Current": t2_EC1_uS_per_cm,
                "leg1_t1_current_water_level": final_t1_water_level,
                "leg1_t2_TDS_current": t2_TDS_ppm,
                "leg1_t2_Humidity_current": t2_Humidity,
            }
        }
        , upsert=False)

    if first_run == 1:
        data_id = values.update_one(
            {"id": "ui_values"},
            {
                "$set": {

                    "leg1_t2_current_water_level": final_t2_water_level,  # uncommenting to get the right value for T2

                }
            }
            , upsert=False)
    else:
        pass

    log_action = 'Flags: Move to T2 ' + str(move_to_t2) + ' T2_orchestrator ' + str(
        T2_orchestrator) + ' PH Check ' + str(move_to_pH)
    print(log_action)

    # log_func("Info", "NA", log_action)

    t2_t3_pump_status = list(telemetry.find({"dev_id": "ffffff0025274924"}, {"payload_fields.Op_Mode": 1}))
    t2_t3_pump_status = t2_t3_pump_status[-1]['payload_fields']['Op_Mode']

    print('t2 - t3 pump status = ', t2_t3_pump_status)

    t2_t1_pump_status = list(telemetry.find({"dev_id": "ffffffff25274475"}, {"payload_fields.Op_Mode": 1}))
    t2_t1_pump_status = t2_t1_pump_status[-1]['payload_fields']['Op_Mode']
    print('t2 - t1 pump status = ', t2_t1_pump_status)

    t1_t2_pump_status = list(telemetry.find({"dev_id": "ffffffff25274919"}, {"payload_fields.Op_Mode": 1}))
    t1_t2_pump_status = t1_t2_pump_status[-1]['payload_fields']['Op_Mode']

    print('t1 - t2 pump status = ', t1_t2_pump_status)
    print('Temp T1 Solenoid Valve', temp_t1_solenoid_valve)

    # Orchestration function starts
    if t1_solenoid_status == 0 and t1_valve_command_sent == 1:
        print('t1_valve_command_sent is off')
        t1_valve_command_sent = 0
    else:
        pass

    if final_t1_water_level > leg1_t1_refill and t1_solenoid_status == 0 and move_to_t2 == 0:
        print('Entering first If')
        move_to_t2 = 1
        T2_orchestrator = 1  # adding this line so that t2_orchestrator stays alive in the middle of the program

    elif final_t1_water_level > leg1_t1_refill and (
            temp_t1_solenoid_valve == 1 or t1_solenoid_status == 1) and t1_valve_command_sent == 0:
        print('checking static waterflow')
        if current_t1_water_level - previous_3_t1_water_level == 0:
            data_id = values.update_one(
                {"id": "ui_values"},
                {
                    "$set": {
                        "t1_t2_orchestrator_status": "Refill over ",
                    }
                }
                , upsert=False)
            log_action = "Closing T1 Solenoid valve"
            log_trigger = "static waterflow detected"
            log_func("Tank 1", log_trigger, log_action)
            t1_solenoid_valve_off()
            t1_valve_command_sent = 1
            temp_t1_solenoid_valve = 0
            move_to_t2 = 1
            T2_orchestrator = 1  # adding this line so that t2_orchestrator stays alive in the middle of the program

            print('static waterflow detected, Closing Solenoid valve')
        else:
            pass

    elif final_t1_water_level <= leg1_t1_refill and temp_t1_solenoid_valve == 0 and t1_t2_pump_status == 0 and t1_solenoid_status == 0:

        log_trigger = "Water in tank 1 below refill level"
        log_action = "Opening the T1 Solenoid valve to refill"
        log_func("Tank 1", log_trigger, log_action)

        t1_solenoid_valve_on()  # (on)
        temp_t1_solenoid_valve = 1
        T2_orchestrator = 0
        if current_t1_water_level - previous_3_t1_water_level == 0:
            data_id2 = values.update_one(
                {"id": "ui_values"},
                {
                    "$set": {
                        "t1_t2_orchestrator_status": "Idle - Waiting for Water refill in T1 "

                    }
                }
                , upsert=False)
        else:
            data_id = values.update_one(
                {"id": "ui_values"},
                {
                    "$set": {
                        "t1_t2_orchestrator_status": "Refilling Tank 1",
                    }
                }
                , upsert=False)
        print('opening the valve to refill, stay open till ball float is reached')


    elif move_to_t2 == 1 and t1_solenoid_status == 0 and final_t1_water_level >= 30 and temp_t1_t2_pump_status == 0 and (
            t2_t3_counter > 0 or first_run == 1):
        t1_t2_pump_status = list(telemetry.find({"dev_id": "ffffffff25274919"}, {"payload_fields.Op_Mode": 1}))
        t1_t2_pump_status = t1_t2_pump_status[-1]['payload_fields']['Op_Mode']
        print('Entering T1 - T2 condition ')
        t2_config = list(values.find({}))
        leg1_water_flow_to_t2 = t2_config[-1]['leg1_water_flow_to_t2']

        leg1_pump_1_delay = t2_config[-1]['leg1_pump_1_delay']
        #set_pump_query_parameters()
        # final_t2_water_level = t2_config[-1][leg1_t2_current_water_level]
        if first_run == 1:
            log_trigger = "First run, move water to T2"
        else:
            log_trigger = "Water moved from T2-T3, so replenishing water in T2 from T1 "
        log_action = "Starting T1 - T2 pump "
        log_func("Tank 1", log_trigger, log_action)

        data_id = values.update_one(
            {"id": "ui_values"},
            {
                "$set": {
                    "t1_t2_orchestrator_status": "Moving water from T1-T2",
                }
            }
            , upsert=False)

        t1_to_t2_pump_on()
        t2_t3_counter = 0
        temp_t1_t2_pump_status = 1
        temp_t1_pump_flag = 1  # to enable not draining t1
        water_flown_to_t2 = water_flown_to_t2 + leg1_water_flow_to_t2
        data_id = values.update_one(
            {"id": "ui_values"},
            {
                "$set": {
                    "leg1_t2_current_water_level": final_t2_water_level + leg1_water_flow_to_t2,
                }
            }
            , upsert=False)
        first_run = 0
        time.sleep(leg1_t1_t2_pump_delay + 120)
        T2_orchestrator = 1
        move_to_pH = 1
        move_to_t2 = 0
    elif move_to_t2 == 1 and t1_solenoid_status == 0 and final_t1_water_level >= 30 and t1_t2_pump_status == 1 and temp_t1_pump_flag == 1:
        temp_t1_t2_pump_status = 0
        temp_t1_pump_flag = 0
        print('changing flag for continuous flow')

    elif T2_orchestrator == 1 and t1_t2_pump_status == 0 and final_t2_water_level > 30:
        # running t2_t1 pump by checking whether t3 is off and turning on t1_t2 along with t2_t1 - recirculation

        print('T2 Orchestrator Starts')
        log_action = "T2 Orchestrator Starts"
        # log_func("Tank 2", "NA", log_action)

        t2_Humidity = list(telemetry.find({"dev_id": "lst25275916"}, {"payload_fields.Humidity": 1}))
        t2_Humidity = float(t2_Humidity[-1]['payload_fields']['Humidity'])
        # print('t2_Humidity = ', t2_Humidity)
        t2_Temperature = list(telemetry.find({"dev_id": "lst25275916"}, {"payload_fields.Temperature": 1}))
        t2_Temperature = float(t2_Temperature[-1]['payload_fields']['Temperature'])
        # print('t2_Temperature = ', t2_Temperature)
        t2_t3_pump_status = list(telemetry.find({"dev_id": "ffffff0025274924"}, {"payload_fields.Op_Mode": 1}))
        t2_t3_pump_status = t2_t3_pump_status[-1]['payload_fields']['Op_Mode']
        print('t2 - t3 pump status = ', t2_t3_pump_status)
        t2_t1_pump_status = list(telemetry.find({"dev_id": "ffffffff25274475"}, {"payload_fields.Op_Mode": 1}))
        t2_t1_pump_status = t2_t1_pump_status[-1]['payload_fields']['Op_Mode']
        print('t1 - t2 pump status = ', t2_t1_pump_status)
        leg1_t2_Humidity_max = list(values.find({}))
        leg1_t2_Humidity_max = float(leg1_t2_Humidity_max[-1]['leg1_t2_Humidity_max'])
        # print('leg1_t2_Humidity_max = ', leg1_t2_Humidity_max)
        leg1_t2_Humidity_min = list(values.find({}))
        leg1_t2_Humidity_min = float(leg1_t2_Humidity_min[-1]['leg1_t2_Humidity_min'])
        # print('leg1_t2_Humidity_min = ', leg1_t2_Humidity_min)
        leg1_t2_Temperature_max = list(values.find({}))
        leg1_t2_Temperature_max = float(leg1_t2_Temperature_max[-1]['leg1_t2_Temperature_max'])
        # print('leg1_t2_Temperature_max = ', leg1_t2_Temperature_max)

        leg1_t2_Temperature_min = list(values.find({}))
        leg1_t2_Temperature_min = float(leg1_t2_Temperature_min[-1]['leg1_t2_Temperature_min'])
        # print('leg1_t2_Temperature_min = ', leg1_t2_Temperature_min)
        t2_t3_pump_status = list(telemetry.find({"dev_id": "ffffff0025274924"}, {"payload_fields.Op_Mode": 1}))
        t2_t3_pump_status = t2_t3_pump_status[-1]['payload_fields']['Op_Mode']
        print('t2 - t3 pump status = ', t2_t3_pump_status)

        if final_t2_water_level <= 30:
            T2_orchestrator = 0
            move_to_t2 = 1

        elif temp_t2_t3_pump_status == 1:
            print('waiting for 2 minutes')
            time.sleep(120)
            t2_t3_pump_status = list(telemetry.find({"dev_id": "ffffff0025274924"}, {"payload_fields.Op_Mode": 1}))
            t2_t3_pump_status = t2_t3_pump_status[-1]['payload_fields']['Op_Mode']
            if t2_t3_pump_status == 0:
                temp_t2_t3_pump_status = 0
                temp_t1_t2_status = 0
                move_to_pH = 1
                temp_t1_t2_pump_status = 0  # adding this to let recirculation happen in the middle of the program
            else:
                pass

        elif move_to_pH == 1 and t2_t1_pump_status == 0 and t2_pH > leg1_t2_pH_min and t2_pH < leg1_t2_pH_max :
            print('t2-t1 pump off')
            t2_pH = list(telemetry.find({"dev_id": "ffffffff25274475"}, {"payload_fields.pH": 1}))
            t2_pH = float(t2_pH[-1]['payload_fields']['pH'])
            print('t2 pH = ', t2_pH)
            t2_TDS_ppm = list(telemetry.find({"dev_id": "ffffff0025274924"}, {"payload_fields.TDS_ppm": 1}))
            t2_TDS_ppm = float(t2_TDS_ppm[-1]['payload_fields']['TDS_ppm'])
            print('t2 TDS_ppm= ', t2_TDS_ppm)
            t2_EC1_uS_per_cm = list(telemetry.find({"dev_id": "ffffff0025274924"}, {"payload_fields.EC_mS_per_cm": 1}))
            t2_EC1_uS_per_cm = float(t2_EC1_uS_per_cm[-1]['payload_fields']['EC_mS_per_cm'])
            print('t2 EC1_uS_per_cm= ', t2_EC1_uS_per_cm)
            t2_t3_pump_status = list(telemetry.find({"dev_id": "ffffffff25274475"}, {"payload_fields.Op_Mode": 1}))
            t2_t3_pump_status = t2_t3_pump_status[-1]['payload_fields']['Op_Mode']
            print('t2 - t3 pump status = ', t2_t3_pump_status)
            t1_t2_pump_status = list(telemetry.find({"dev_id": "ffffffff25274919"}, {"payload_fields.Op_Mode": 1}))
            t1_t2_pump_status = t1_t2_pump_status[-1]['payload_fields']['Op_Mode']
            print('t1 - t2 pump status = ', t1_t2_pump_status)
            solenoid_t3_drain_status_raw = list(telemetry.find({"dev_id": "a840410001818dab"}, {"payload_fields.Operating_Mode": 1}))
            t3_drain_solenoid_status = solenoid_t3_drain_status_raw[-1]['payload_fields']['Operating_Mode']
            #print('t3 to drain solenoid status = ', t3_drain_solenoid_status)
            if t2_t3_pump_status == 0 and temp_t2_t3_pump_status == 0 and t1_t2_pump_status == 0 and t3_drain_solenoid_status == 0:
                print('T2 and T3 pumps off')
                print('ph min', leg1_t2_pH_min)
                print('ph max', leg1_t2_pH_max)
                #set_pump_query_parameters()
                config1 = list(values.find({}))
                log_trigger = "pH conditions met "
                log_action = "Starting T2-T3 pump"
                log_func("Tank 2", log_trigger, log_action)

                t2_t3_pump_on()
                water_flown_to_t3 = leg1_water_flow_t2_t3
                T2_orchestrator = 1
                final_t2_water_level = config1[-1]['leg1_t2_current_water_level']
                print('final_t2_water_level', final_t2_water_level)
                print('water_flown_to_t3', water_flown_to_t3)
                final_t2_water_level = final_t2_water_level - water_flown_to_t3
                print('water_level_after_sending to t3', final_t2_water_level)
                temp_t2_t3_pump_status = 1
                t2_t3_counter += 1
                recirculation_flag = 0
                print('T2-T3 counter = ', t2_t3_counter)
                log_action = "T2-T3 counter" + str(t2_t3_counter)
                # log_func("Tank 2", "NA", log_action)

                data_id2 = values.update_one(
                    {"id": "ui_values"},
                    {
                        "$set": {
                            "water_flown_to_t3": water_flown_to_t3,
                            "leg1_t2_current_water_level": final_t2_water_level,
                            "t1_t2_orchestrator_status": " Moving water from T2-T3"

                        }
                    }
                    , upsert=False)
                print('T2-T3 pump turned on')
                # move_to_farm_drain = 1
                move_to_pH = 0
                time.sleep(120)

        elif t2_t3_counter > 0:
            # print('All conditions failed 1')
            log_action = "pH condition not met"
            # log_func("Tank 2", "NA", log_action)
            print('T2-T3 Counter = ', t2_t3_counter)
            T2_orchestrator = 0
            print('T2 Orchestrator is off')
            log_action = "T2 Orchestrator is off"
            log_func("Tank 2", "NA", log_action)


        elif t2_t3_pump_status == 0 and t2_Temperature > leg1_t2_Temperature_min and t2_Temperature < leg1_t2_Temperature_max and t2_Humidity > leg1_t2_Humidity_min and t2_Humidity < leg1_t2_Humidity_max:
            config = list(values.find({}))
            recirculation_gap_time = config[-1]['recirculation_gap_time']
            if recirculation_flag == 1:
                print('waiting for recirculation gap time')
                time.sleep(recirculation_gap_time)
            else:
                recirculation_flag = 0

            if t1_t2_pump_status == 0 and t2_t1_pump_status == 0:
                temp_t1_t2_pump_status = 0
                temp_t1_t2_status = 0
                temp_t2_t3_pump_status = 0
            else:
                print('waiting for recirculation pump to turn off')
                time.sleep(180)
            print('changing recirculation temp flags to zero since orchestrator has nothing better to do')
            if temp_t1_t2_status == 0 and temp_t1_t2_pump_status == 0 and temp_t2_t3_pump_status == 0:
                log_trigger = "Recirculation conditions met"
                log_action = "Starting T2-T1 pump"
                log_func("Tank 2", log_trigger, log_action)
                data_id = values.update_one(
                    {"id": "ui_values"},
                    {
                        "$set": {
                            "t1_t2_orchestrator_status": "Recirculating water",
                        }
                    }
                    , upsert=False)
                t2_t1_pump_on()

                # print('T2-T1 Pump is on')
                # water_flown_to_t1 = water_flown_to_t1 + leg1_water_flow_t2_t1
                temp_t2_t1_status = 1
                print('recirculation starts')

                while True:
                    t2_t1_pump_status = list(
                        telemetry.find({"dev_id": "ffffffff25274475"}, {"payload_fields.Op_Mode": 1}))
                    t2_t1_pump_status = t2_t1_pump_status[-1]['payload_fields']['Op_Mode']
                    if t2_t1_pump_status == 1 and temp_t1_t2_status == 0 and temp_t2_t1_status == 1:
                        #set_pump_query_parameters()

                        log_trigger = "T2-T1 recirculation started"
                        log_action = "Starting T1-T2 pump"
                        log_func("Tank 2", log_trigger, log_action)

                        t1_t2_pump_on()
                        temp_t1_t2_status = 1
                        # water_flown_to_t2 = water_flown_to_t2 + leg1_water_flow_t2_t1
                        print('T1-T2 pump is on')
                        print('recirculation at full power')
                        recirculation_flag = 1
                        break

            else:
                data_id2 = values.update_one(
                    {"id": "ui_values"},
                    {
                        "$set": {
                            "t1_t2_orchestrator_status": "Idle - No condition for Trigger in T2 "

                        }
                    }
                    , upsert=False)
                print('All conditions failed 2')
        else:
            print('All conditions failed 3')
            log_trigger = "pH condition not met & Temperature and Humidity not met"
            log_func("Tank 2", log_trigger, "Idle")
            data_id2 = values.update_one(
                {"id": "ui_values"},
                {
                    "$set": {
                        "t1_t2_orchestrator_status": "Idle - Waiting for Trigger in T2 "

                    }
                }
                , upsert=False)
    continue









Loop time 2020-04-06 09:36:37.014629  run count  2
t3 to drain solenoid status =  0
T1 Solenoid Status =  0
('T1 - ', 40.0, ' T2 - ', 166.0)
Flags: Move to T2 0 T2_orchestrator 0 PH Check 0
t2 - t3 pump status =  0
t2 - t1 pump status =  0
t1 - t2 pump status =  0
Temp T1 Solenoid Valve 0
Entering first If

Loop time 2020-04-06 09:36:51.144820  run count  3
t3 to drain solenoid status =  0
T1 Solenoid Status =  0
('T1 - ', 40.0, ' T2 - ', 166.0)
Flags: Move to T2 1 T2_orchestrator 1 PH Check 0
t2 - t3 pump status =  0
t2 - t1 pump status =  0
t1 - t2 pump status =  0
Temp T1 Solenoid Valve 0
Entering T1 - T2 condition 

Loop time 2020-04-06 09:39:09.664791  run count  4
t3 to drain solenoid status =  0
T1 Solenoid Status =  0
('T1 - ', 37.0, ' T2 - ', 169.0)
Flags: Move to T2 0 T2_orchestrator 1 PH Check 1
t2 - t3 pump status =  0
t2 - t1 pump status =  0
t1 - t2 pump status =  1
Temp T1 Solenoid Valve 0
Entering first If

Loop time 2020-04-06 09:39:19.283374  run count  5
t3 to drain

KeyboardInterrupt: ignored